In [20]:
import pandas as pd
import json
import re
import pymongo
import jieba
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.decomposition import LatentDirichletAllocation as LDA
from gensim import corpora, models, similarities

ModuleNotFoundError: No module named 'gensim'

In [2]:
#連接 mongodb

client = pymongo.MongoClient(host='123.241.175.34', port=27017)
#mongodb帳號密碼

client.admin.authenticate('root', '1qaz@WSX3edc')

True

In [3]:
#設定名稱為test的資料庫 也可用 db = client['test']
client.database_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  


['ETL', 'admin', 'config', 'local', 'raw_article', 'test']

In [3]:
db_ETL=client.ETL

In [4]:
#顯示db中所有collections
db_ETL.list_collection_names()

['article_tag_soc',
 'article_tag_news',
 'article_news_seg',
 'article_tag_web',
 'article_web_seg',
 'article_social_seg']

In [5]:
article_social=db_ETL.article_social_seg.find()
article_web=db_ETL.article_web_seg.find()
article_news=db_ETL.article_news_seg.find()

In [6]:
df_article_social=pd.DataFrame(list(article_social))
df_article_web=pd.DataFrame(list(article_web))
df_article_news=pd.DataFrame(list(article_news))

In [7]:
article_social_seg=df_article_social['seg']
article_web_seg=df_article_web['seg']
article_news_seg=df_article_news['seg']

In [8]:
tf_article=[]
punc="!#$%&\'()*+,-./:;<=>?@，。?～（｜＋★▉、＝…\ ●【】《》 ）＊？“”‘’！[\\]^_`{|}~]+'\#"
gar=['\\t','\n','\t ']
for i in range(len(article_web_seg)):
    word=''
    for w  in article_web_seg[i]:
        if w not in punc and gar:
            word+=w+' '
    tf_article.append(word)

In [9]:
tf_article[:10]

['玉山 Pi 拍錢包 信用卡 國內 4.5% 海外 3% 繳稅 1% 持續 坐 穩神 卡寶座 May16Thu201923 12 玉山 Pi 拍錢包 信用卡 國內 4.5% 海外 3% 繳稅 1% 持續 坐 穩神 卡寶座 玉山 Pi 拍錢包 信用卡 從 去年 三季 推出 除了 基本 回饋 最高 4.5% 繳水電 也 有 回饋 的 強大 優勢 且 卡片 限定 短期 優惠 也 能 有效 提升 動卡率 讓 人 感覺 銀行 十足 誠 意在 經營 這張 卡片 也 因此 這 一年 來 討論度 不降反升 熱度 依舊 不管 到 哪個 信用卡 的 論壇 搜尋關鍵 字 每天 都 有人 在 分享 Pi 拍錢包 信用卡 分享 數多到 要 花 好 幾個 小時 消化 完 這張 卡 也 成為 我 跟 讀者 私訊 討論 次數 最 多 的 卡片 前 五名 有 網友 曾經 跟 我 討論 到 半夜 兩點 這個 經驗 真的 Neveragain 由 於 Pi 拍錢包 信用卡 也 是 我 今年 上半年 的 愛用 卡 之一 我會 透過 這 篇文章 整合 卡片 特性 使用 經驗 跟 評價 依序 幫 大家 整理 幾個 重點 ： 玉山 Pi 拍錢包 信用卡 是 什麼 什麼 類型 的 人 申辦 申辦 前須 知道 的 重要 事項 玉山 Pi 拍錢包 信用卡 重點 權益 近期 超強勢 必用 回饋 點數 折抵 攻略 Pi 拍錢包 信用卡 是 一張 主打 點數 回饋 的 卡片 回饋 的 幣別 叫做 Ｐ 幣的 有 價點數 而 1 點 P 幣 1 元現 金 點數 可以 在 指定 特約 店家 折 抵消 費 有 的 人覺 得 還是 現金回饋 更 直接 實用 但 Pi 拍錢包 合作 的 店家 有 非常 多 知名 連鎖 品牌 並且 吃喝 玩樂 都 有 兼 顧到 因此 能 折抵 的 機會 很多 有 的 人 覺得點 數折 抵消 費很麻煩 近期 上線 的 新 功能 「 拍 享券 」 能 幫助 卡友 輕 鬆 折抵 這個 新 功能 我 在 文章 後 面一點 會 提到 如果 覺得 點數 兌換 很 麻煩 一定 要 了解 這個 功能 Pi 拍錢包 信用卡 卡面 設計 有 四個 顏色 通常 較 多 人會 選擇 申辦 藍色 的 卡面 因為 有 附加 悠遊卡 的 功能 我 都 帶 著 這張 卡去 超商 消費 或 搭捷 運公車 再者 我 推薦 黑色 霧面 的

In [10]:
vectorizer = TfidfVectorizer(max_features=None,sublinear_tf=True,use_idf=True,max_df=0.8,min_df=20,ngram_range=(1,4))
tfidf=vectorizer.fit_transform(tf_article)

In [11]:
tfidf_array=tfidf.toarray()

In [60]:
tfidf_array.shape

(2830, 37042)

In [27]:

tf_feature_names =vectorizer.get_feature_names()

In [32]:
# Tweak the two parameters below
number_topics = 10
number_words = 10

# Print the topics found by the LDA model
print("Topics found via LDA:")
#print_topics(lda, count_vectorizer, number_words)

lda = LDA(n_components=15, n_jobs=-1)
lda.fit(tfidf)
# Print the topics found by the LDA model
print("Topics found via LDA:")
#print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:
Topics found via LDA:


In [22]:
print(lda.fit(tfidf))

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)


In [34]:
def show_topics(vectorizer=vectorizer, lda_model=lda, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [33]:
n_top_words=50
print_top_words(lda, tf_feature_names, n_top_words)

NameError: name 'print_top_words' is not defined

In [35]:
show_topics(vectorizer,lda,50)

[array(['借錢', '貸款 利率', '信貸', '借貸', '借款', '貸款', '民間', '代書', '不良', '貸款 代辦',
        '小額', '潮霖', '潮霖 資產', '地下', '信用 不良', '融資', '資產', '二胎', '信用 貸款',
        '忠訓', '錢莊', '地下 錢莊', '合法', '土地', 'ok 忠訓', '房貸 利率', '有限公司', '勞工',
        '汽車 貸款', '資產 有限公司', '軍人', '潮霖 資產 有限公司', '評價 如何', '理債 一日', '利率 比較',
        '貸款 潮霖', '勞工 貸款', '率利', '代辦', '房屋 貸款', '房貸', '銀行 貸款', '行房', '理債',
        '高利', '房屋', '貸款 率利', '一日', '速配', 'ok'], dtype='<U42'),
 array(['信用 利率', '循環 信用 利率', '每筆 預借 現金', '每筆 預借 現金 手續', '循環 信用',
        '利率 68 15', '利率 68 15 00', '100 預借現', '100 預借現 金金', '循環 信用 利率 68',
        '利率 68', '信用 利率 68', '68 15', '68 15 00', '信用 利率 68 15',
        '104 每筆 預借', '基準 104 每筆', '基準 104 每筆 預借', '104 每筆 預借 現金',
        '其他 費用 請洽', '68 15 00 循環', '15 00 循環', '費用 請洽', '104 每筆',
        '15 00 循環 利率', '00 循環', '利率 基準 104 每筆', '00 循環 利率', '00 循環 利率 基準',
        '信用卡 循環 信用 利率', 'nt 100', '15 00', '現金為', '循環 利率', '差別 循環 信用',
        '差別 循環', '依電腦', '消費及 預借 現金為', '不足 nt 100', '依電腦 評等', '循環 信用 利率 一般',
        '不足